In [1]:
import pandas as pd

In [2]:
fc_file = '../featureCounts/all_feature.txt'
df = pd.read_csv(fc_file,header=1,sep='\t')
df_geneid = df.iloc[:,0:1]
df_counts = df.iloc[:,6:]
df_merge = pd.concat([df_geneid,df_counts],axis=1)
df_merge

,Geneid,../bam/KD_NC-rep1_Aligned_sort_rmdup.bam,../bam/KD_NC-rep2_Aligned_sort_rmdup.bam,../bam/KD_NC-rep3_Aligned_sort_rmdup.bam,../bam/KD_CCNB1-rep1_Aligned_sort_rmdup.bam,../bam/KD_CCNB1-rep2_Aligned_sort_rmdup.bam,../bam/KD_CCNB1-rep3_Aligned_sort_rmdup.bam,../bam/KD_PUS7-rep1_Aligned_sort_rmdup.bam,../bam/KD_PUS7-rep2_Aligned_sort_rmdup.bam,../bam/KD_PUS7-rep3_Aligned_sort_rmdup.bam
0,NR_046018.2,0,0,1,0,0,0,0,0,0
1,NR_024540.1,602,520,415,436,402,464,404,442,466
2,NR_106918.1,16,7,6,11,4,12,7,14,13
3,NR_036051.1,0,0,0,0,0,0,0,0,0
4,NR_026818.1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
55963,YP_003024034.1,9324,9779,8780,12293,11196,12425,9399,9839,9938
55964,YP_003024035.1,56741,59157,50403,79958,72706,79907,56552,61648,61619
55965,YP_003024036.1,38441,39733,28731,51552,50592,57086,35407,42761,42040
55966,YP_003024037.1,11938,11094,8345,14343,15016,16335,10045,12068,11917


In [3]:
colName = ['Geneid','NC1','NC2','NC3','CCNB11','CCNB12','CCNB13','PUS71','PUS72', 'PUS73']

In [4]:
# df = df.copy()
df_merge.columns=colName
df_merge

,Geneid,NC1,NC2,NC3,CCNB11,CCNB12,CCNB13,PUS71,PUS72,PUS73
0,NR_046018.2,0,0,1,0,0,0,0,0,0
1,NR_024540.1,602,520,415,436,402,464,404,442,466
2,NR_106918.1,16,7,6,11,4,12,7,14,13
3,NR_036051.1,0,0,0,0,0,0,0,0,0
4,NR_026818.1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
55963,YP_003024034.1,9324,9779,8780,12293,11196,12425,9399,9839,9938
55964,YP_003024035.1,56741,59157,50403,79958,72706,79907,56552,61648,61619
55965,YP_003024036.1,38441,39733,28731,51552,50592,57086,35407,42761,42040
55966,YP_003024037.1,11938,11094,8345,14343,15016,16335,10045,12068,11917


In [5]:
df_merge.to_csv(fc_file.replace('.txt','_fix.csv'),index=None,sep='\t')